# Working with the dataset

Loading the dataset:

In [53]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [54]:
# See the size of the dataset:
print(len(text))

# Print the first 1000 characters of the text:
print(text[:1000])

1115393
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

Creating the vocabulary:

In [55]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


**Tokenization**

In [56]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode("Hello World!"))
print(decode(encode("Hello world!")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello world!


*Note* - This is character-wise encoding-decoding, but here are examples of others, more complex, approaches:
- SentencePiece (Google) - https://github.com/google/sentencepiece
- TikToken (OpenAI) - https://github.com/openai/tiktoken

Encoding the entire dataset text:

In [57]:
import torch

data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape, data.dtype)
print(data[0:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Train/Validation data split:

In [58]:
n = int(len(data) * 0.9)
train_data, val_data = data[:n], data[n:]

In [59]:
context_length = 8
train_data[0:context_length + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

Example of training context and target:

In [60]:
x = train_data[:context_length]
y = train_data[1:context_length + 1]

for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    print(f'context: {context}, targe: {target}')

context: tensor([18]), targe: 47
context: tensor([18, 47]), targe: 56
context: tensor([18, 47, 56]), targe: 57
context: tensor([18, 47, 56, 57]), targe: 58
context: tensor([18, 47, 56, 57, 58]), targe: 1
context: tensor([18, 47, 56, 57, 58,  1]), targe: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]), targe: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]), targe: 58


Generalising for batch_size>1:

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(split='train'):
    data = train_data if split=='train' else val_data                   # choosing the data from where to take the example
    ix = torch.randint(0, len(data) - context_length, (batch_size,))    # choosing the starting index of the example
    x = torch.stack([ data[i:i+context_length] for i in ix ])           # creating the context tensor
    y = torch.stack([ data[i+1:i+context_length+1] for i in ix])        # creating the target tensor
    x, y = x.to(device), y.to(device)                                   # moving the tensors to the device
    return x, y

xb, yb = get_batch()
print('inputs:')
print(xb.shape)
print(xb)
print('outputs:')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(context_length):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'context: {context}, target: {target}')

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
outputs:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
context: tensor([53]), target: 59
context: tensor([53, 59]), target: 6
context: tensor([53, 59,  6]), target: 1
context: tensor([53, 59,  6,  1]), target: 58
context: tensor([53, 59,  6,  1, 58]), target: 56
context: tensor([53, 59,  6,  1, 58, 56]), target: 47
context: tensor([53, 59,  6,  1, 58, 56, 47]), target: 40
context: tensor([53, 59,  6,  1, 58, 56, 47, 40]), target: 59
context: tensor([49]), target: 43
context: tensor([49, 43]), target: 43
context: tensor([49, 43, 43]), target: 54
context: tensor([49, 43, 43, 54]), target: 1
context: tensor([49, 43, 43, 54,  1]), target: 47
context: tensor(

# Neural networks

### Bigram Language Model

In [62]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # shapes of idx and targets are [batch_size, context_size]
        logits = self.token_embedding_table(idx) # shape: [batch_size, context_size, vocab_size]

        # F.cross_entropy expects batch_size x CHANNELS x d_2 x d_3...
        if targets != None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)    
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # shape: [batch_size, vocab_size, context_size]
        else:
            loss = None

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)
            logits = logits[:, -1, :]                           # shape: [batch_size, vocab_size] (selecting the last token)
            probs = F.softmax(logits, dim=-1)                   # shape: [batch_size, vocab_size]
            idx_next = torch.multinomial(probs, num_samples=1)  # shape: [batch_size, 1]
            idx = torch.cat([idx, idx_next], dim=1)             # shape: [batch_size, context_size+1]
        return idx
    
bigram_model = BigramLanguageModel(vocab_size)
logits, loss = bigram_model(xb, yb)
print(logits.shape, loss)

print(decode(bigram_model.generate(torch.zeros((1, 1), dtype=torch.long), 100)[0].tolist()))


torch.Size([32, 65]) tensor(4.8948, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


Training the Bigram Language Model:

In [ ]:
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device('cpu')

optimizer = torch.optim.AdamW(bigram_model.parameters(), lr=1e-3)

In [ ]:
batch_size = 32

bigram_model = bigram_model.to(device)

for steps in range(10000):
    # sampling a batch of data
    xb, yb = get_batch()

    # evaluate the loss
    logits, loss = bigram_model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f'loss: {loss.item()}')

loss: 2.4782700538635254


In [40]:
# Test the model:
print(decode(bigram_model.generate(torch.zeros((1, 1), dtype=torch.long), 300)[0].tolist()))


NUMPA t hofr tiWhyst o suilout 'de LBererre tssu PAn my tharr gs moona pedealabes;
Thew m.
Whifoff atoultho?
KI t ORipo gleloungithistt:
ng, bawne,
By
A:
RDut t?
pwanovead her t t APAUESLANEd titrakerath athed s,
RI lk.
Anordenare pineFFadl wh,
T:


e t,


Wheathas wouns Ply
MI lif corndiver ve. he 


# Self Attention

Mathematical trick in self-attention:

In [65]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

### 1. Simple form of attention (Average of previous tokens)

**Version 1** - Inneficient way to do it:

In [66]:
xbow = torch.zeros((B, T, C)) # bows stands for bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # Shape [t, C]
        xbow[b, t] = torch.mean(xprev, dim=0)

print(x[0])
print(xbow[0])


tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


**Version 2** - Efficient way to do it:

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))            # lower triangular matrix with ones
a = a / torch.sum(a, dim=1, keepdim=True)   # normalizing the rows to sum to 1 (we are calculating the mean)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

tensor([[1.],
        [2.],
        [3.]])
tensor([1., 2., 3.])
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [76]:
weights = torch.tril(torch.ones(T, T))
weights = weights / torch.sum(weights, dim=1, keepdim=True)
xbow2 = weights @ x
print(xbow2[0])
print(torch.allclose(xbow, xbow2))

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
True


**Version 3** - How it's done in practice:

In [77]:
tril = torch.tril(torch.ones(T, T))
weights3 = torch.zeros((T, T))
weights3 = weights3.masked_fill(tril==0, float('-inf'))
weights3 = F.softmax(weights3, dim=-1)
xbow3 = weights3 @ x
torch.allclose(xbow, xbow3)

True

### Self Attention Head

In [ ]:
torch.manual_seed(1337)

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T, T))

# each token emits a query and a key vector
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      # shape: [B, T, head_size]
q = query(x)    # shape: [B, T, head_size]

# computing the attention weights
weights = q @ k.transpose(-2, -1)       # shape: [B, T, head_size] @ [B, head_size, T] ---> [B, T, T]
weights = weights * head_size**-0.5     # SCALED SELF-ATTENTION
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=-1)

v = value(x)    # shape: [B, T, head_size]
attention_output = weights @ v          # shape: [B, T, T] @ [B, T, head_size] ---> [B, T, head_size]

print(attention_output.shape)

torch.Size([4, 8, 16])


*Notes*:
- Attention is a communication mechanism. Can be applied to any graph.
- There is no notion of space. Nodes are not informed about their (or other) positions in the sequence.
- **Self-Attention** <=> keys, queries and the values are coming from the same source
- **Cross-Atttention** <=> (queries) from one source, while (keys, values) from another source
- *Scaled attention* --> This is done to normalise the variance of the weights (avoiding softmax converging toward one-hot vectors)